In [4]:
!nvidia-smi

Fri Apr 22 11:16:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import numpy as np
import pandas as pd
import os
import keras.metrics
import matplotlib.pyplot as plt
from imutils import paths
from sklearn.model_selection import train_test_split

In [7]:
test_dir =  '/content/drive/MyDrive/Major Project/Datasets/Plant Disease Dataset/plantvillage dataset/color/'

In [8]:
fruits = []
fruits_image = []
for i in os.listdir(test_dir):
    for image_filename in os.listdir(test_dir + i):
        fruits.append(i) # name of the fruit 
        fruits_image.append(i + '/' + image_filename)

In [9]:
test_fruits = pd.DataFrame(fruits, columns=["Fruits"])
test_fruits["Fruits Image"] = fruits_image

In [10]:
test_fruits

,Fruits,Fruits Image
0,Apple___Apple_scab,Apple___Apple_scab/01a66316-0e98-4d3b-a56f-d78...
1,Apple___Apple_scab,Apple___Apple_scab/00075aa8-d81a-4184-8541-b69...
2,Apple___Apple_scab,Apple___Apple_scab/01f3deaa-6143-4b6c-9c22-620...
3,Apple___Apple_scab,Apple___Apple_scab/0a5e9323-dbad-432d-ac58-d29...
4,Apple___Apple_scab,Apple___Apple_scab/2e264ee0-7679-4ad2-a19a-1ac...
...,...,...
54300,Tomato___healthy,Tomato___healthy/271be296-4c0b-4817-a44b-cc961...
54301,Tomato___healthy,Tomato___healthy/1b150c8d-ab5d-43b5-a61d-c3fb3...
54302,Tomato___healthy,Tomato___healthy/0a86ddc0-4950-4efc-9260-0f16d...
54303,Tomato___healthy,Tomato___healthy/4022afd0-756b-4f07-9a8a-3cdf4...


In [11]:
image_paths=[]
data_labels=[]

In [12]:
for i in range(len(test_fruits)):
  path=test_fruits['Fruits Image'][i]
  path=test_dir+path
  label=test_fruits['Fruits'][i]
  image_paths.append(path)
  data_labels.append(label)

In [20]:
# Training and Validation split
from sklearn.model_selection import train_test_split
(tdpl, image_paths, tdcl, data_labels) = train_test_split(image_paths,
                                                                                                        data_labels,
                                                                                                        test_size = 0.1,
                                                                                                        stratify = data_labels,
                                                                                                        random_state = 101)

In [21]:
df_test = pd.DataFrame({"Path":image_paths,"Class":data_labels})

In [22]:
df_test

,Path,Class
0,/content/drive/MyDrive/Major Project/Datasets/...,Strawberry___Leaf_scorch
1,/content/drive/MyDrive/Major Project/Datasets/...,"Pepper,_bell___healthy"
2,/content/drive/MyDrive/Major Project/Datasets/...,Cherry_(including_sour)___Powdery_mildew
3,/content/drive/MyDrive/Major Project/Datasets/...,Soybean___healthy
4,/content/drive/MyDrive/Major Project/Datasets/...,Soybean___healthy
...,...,...
5426,/content/drive/MyDrive/Major Project/Datasets/...,Tomato___Late_blight
5427,/content/drive/MyDrive/Major Project/Datasets/...,"Pepper,_bell___healthy"
5428,/content/drive/MyDrive/Major Project/Datasets/...,Apple___Black_rot
5429,/content/drive/MyDrive/Major Project/Datasets/...,Soybean___healthy


In [23]:
from keras.models import load_model

trained_model = load_model("/content/drive/MyDrive/Major Project/Training Files/Dataset 2/Transfer Learning/Ratio 1/Models/vgg16-ep30.h5")

In [24]:
trained_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0   

In [25]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [28]:
ts_data = ImageDataGenerator()
testing_df = ts_data.flow_from_dataframe(df_test, x_col = "Path",y_col = "Class", batch_size=1, target_size=(128, 128))

Found 5431 validated image filenames belonging to 38 classes.


In [29]:
trained_model.evaluate(testing_df)

5431/5431 [==============================] - 1420s 261ms/step - loss: 0.2177 - accuracy: 0.9405 - precision: 0.9525 - recall: 0.9308 - auc: 0.9959 - tp: 5055.0000 - fp: 252.0000 - tn: 200695.0000 - fn: 376.0000


[0.2177412509918213,
 0.9405266046524048,
 0.9525155425071716,
 0.930767834186554,
 0.9958875179290771,
 5055.0,
 252.0,
 200695.0,
 376.0]